# Self Driving Traffic Sign Detection

Isaiah Jenkins

## Import the required libraries

In [1]:
import tensorflow as tf
import pandas as pd
import os
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split


## 1. About the data

1. a. Description

Throughout this analysis we will explore Udacity's Self Driving dataset. This dataset consists of images for thousands of pedestrians, bikers, cars, and traffic lights. Although traffic light images are underrepresented in the dataset, the focus of the analysis will be based solely on traffic light images.

1. b. Data dictionary, 97,942 labels across 11 classes and 15,000 images, 1,720 null examples (images with no labels).

Class Balance across images (labels in each image)

* car - 64,399 - over represented
* pedestrian - 10,806
* trafficLight-Red - 6,870
* trafficLight-Green - 5,465 - under represented
* truck - 3,623 - under represented
* trafficLight - 2,568 - under represented
* biker - 1,864 - under represented
* trafficLight-RedLeft - 1,751 - under represented
* trafficLight-GreenLeft - 310 - under represented
* trafficLight-Yellow - 272 - under represented
* trafficLight-YellowLeft - 14 - under represented

## 2. Objectives

Throughout this analysis, we will explore and build various deep learning convolutional neural network (CNN) architectures to detect traffic light signs, aiming to optimize accuracy and efficiency. Our objective is to compare different model variations, such as CNNs with different depths, pre-trained models, and data augmentation techniques, to determine the most effective approach. Potential challenges include handling variations in lighting conditions, occlusions, and small object sizes, which may impact detection performance. Additionally, dataset imbalances and misclassifications due to similar-looking traffic signs could introduce biases, requiring careful preprocessing and model tuning.

## 3. Data Exploration, Cleaning and Feature Engineering

* Extract traffic sign & traffic light images from the dataset to help with computational efficiency.
* Resize images to standardize dataset making it computationally efficient making it easier to analyze data.
* Normalize pixel values (0-1 range) to help with generalization.
* Augment data (rotation, brightness shifts, contrast changes) to improve generalization.

### Load in data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!unzip -o -q '/content/gdrive/MyDrive/data.zip' -d '/content/'

Mounted at /content/gdrive


In [13]:
DATASET_PATH = 'data/export'
CSV_PATH = os.path.join(DATASET_PATH, '_annotations.csv')

In [4]:
df = pd.read_csv(CSV_PATH) # load annotations

In [14]:
df = df[df['class'].isin(['trafficLight-Red', 'trafficLight_Yellow', 'trafficLight-Green'])]

In [15]:
file_names = df['filename'].values # image files names
labels = df['class'].values # classes

In [16]:
label_map = {"trafficLight-Red": 0, "trafficLight_Yellow": 1, "trafficLight-Green": 2, }  # Adjust based on dataset
labels = [label_map[label] for label in labels if label in label_map]  # Convert text labels to integers
labels = to_categorical(labels, num_classes=3)  # Convert to one-hot encoding

In [17]:
#Split Data into Train (80%) and Test (20%)
train_files, test_files, train_labels, test_labels = train_test_split(
    file_names, labels, test_size=0.2, random_state=42, stratify=labels, shuffle=True
)

In [18]:
IMG_SIZE = (224, 224)

In [19]:
# Function to Load & Preprocess Images
def load_and_preprocess_image(file_name, label):
    img_path = os.path.join(DATASET_PATH, file_name)  # Update image folder path
    img = load_img(img_path, target_size=IMG_SIZE)  # Load & Resize Image
    img = img_to_array(img) / 255.0  # Convert to NumPy array & Normalize (0-1)
    return img, label

In [20]:
# Create Generators for Train & Test Data
def data_generator(file_list, label_list):
    for f, l in zip(file_list, label_list):
        yield load_and_preprocess_image(f, l)  # Load one image at a time

In [21]:
BATCH_SIZE = 10

In [23]:
# Create a tf.data.Dataset that Loads Images on Demand
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_files, train_labels),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),  # Image Shape
        tf.TensorSpec(shape=(3,), dtype=tf.float32)  # One-hot Encoded Label
    )
).batch(BATCH_SIZE).shuffle(1000).prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_files, test_labels),
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),  # Image Shape
        tf.TensorSpec(shape=(3,), dtype=tf.float32)  # One-hot Encoded Label
    )
).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [24]:
# Preview a Small Batch
for img_batch, label_batch in train_dataset.take(1):
    print(f"Image Batch Shape: {img_batch.shape}")
    print(f"Label Batch Shape: {label_batch.shape}")

Image Batch Shape: (10, 224, 224, 3)
Label Batch Shape: (10, 3)


## 4. CNN Models

### Summary of Models

## 5. Insights and Key Findings

## 6. Next Steps